# Iterative LQG

This notebook implements the ILQG method, as described in the following paper:

[A generalized iterative LQG method for locally-optimal feedback
control of constrained nonlinear stochastic systems](http://maeresearch.ucsd.edu/skelton/publications/weiwei_ilqg_CDC43.pdf)

In general, suppose that we have the following dynamical system:

$$ x_{k+1} = f(x_{k}, u_{k})$$
$$ J_0 = \frac{1}{2} (x_N - x^*)^TQ_f(x_N - x^*) + \frac{1}{2} \sum_{k=0}^{T-1}(x_k^TQx_k + u_k^TRu_k) $$

In [56]:
# let's work with a very simple mdp
from mdp import CartpoleMDP
import numpy as np
mdp = CartpoleMDP()
print 'obs shape:', mdp.observation_shape
print '# actions:', mdp.n_actions
# We'll start simple and suppose that we already have the cost function
Q = np.diag([1, 1, 1, 1])
R = np.diag([1])
# time horizon
N = 100
Da = mdp.n_actions
# initialize with random actions
# u = np.random.randn(N, Da)

state, obs = mdp.sample_initial_state()
# make this a fixed horizon for now, so that the state is not reset when in reality the system has already failed
for _ in range(N):
    action = np.random.randn(Da)
    next_state, next_obs, reward, done, step = mdp.step_single(state, action)
#     print next_state
    state, obs = next_state, next_obs
#     print done
# print u
#nominals = 

obs shape: (4,)
# actions: 1


In [57]:
# The problem that iLQG faces is kind of a different flavor
# from typical RL problems. Here we usually assume a nonlinear dynamical system,
# whose cost can be locally approximated by a positive definite quadratic cost,
# and where the dynamics can be locally linearized.
# Let's first port the matlab version of iLQR to Python. The matlab version was authored
# by Timothy Lillicrap (http://limb.biomed.queensu.ca/lab_members/timothylillicrap/projects.php)

def pendulum_dynamics(x, u, dt):
    g = 9.8
    m = 1.0 
    l = 1.0 # length of the rod
    mu = 0.01 # friction coefficient
    th, thdot = x
    u0 = u[0]
    newth = th + thdot*dt
    newthdot = thdot + ( (-mu/(m*l**2))*thdot + (g/l)*np.sin(th) + u/(m*l**2) ) * dt
    return np.array([newth, newthdot])

print pendulum_dynamics(np.array([np.pi, 0]), np.array([0]), 0.005)

[  3.14159265e+00   6.00076932e-18]


In [58]:
def ilqr_openloop(x0, u, dt, sysdyn):
    N = len(u) + 1
    traj = np.zeros((N, len(x0)))
    traj[0] = x0
    for t in range(1, N):
        traj[t] = sysdyn(traj[t-1], u[t-1], dt)
    return traj

In [59]:
def ilqr_cost(u, x0, xf, Qf, Q, R, dt, sysdyn):
    # run the system under consideration forward using the command signal u
    # and calculate the associated cost defined by xf, Qf, Q, R.
    # u: the control signal with which to control the system
    # x0: the state of the system at some initial time
    # xf: the desired final state of the system
    # Qf: the quadratic final cost matrix
    # Q the quadratic integrated cost matrix on the state
    # R the quadratic integrated cost matrix on the command
    # dt the current time step used
    # sysdyn: dynamics of the system
    # 
    # Return: the associated cost.

    x = ilqr_openloop(x0, u, dt, sysdyn)
    N = len(x)

    cost = 0.5*(x[N-1] - xf).T.dot(Qf).dot(x[N-1] - xf)
    for t in range(N-1):
        cost += 0.5*(x[t].T.dot(Q).dot(x[t]) + u[t].T.dot(R).dot(u[t]))
    return cost

In [60]:
def ilqr_linearization(x, u, dt, sysdyn, eps=1e-5):
    # perform finite-difference on the 
    Nx = len(x)
    Nu = len(u)
    A = np.zeros((Nx, Nx))#Nx, Nx))
    B = np.zeros((Nx, Nu))
    eyex = np.eye(Nx)
    eyeu = np.eye(Nu)
    for dx in range(Nx):
        xp = x + eyex[dx] * eps
        xn = x - eyex[dx] * eps
        A[:, dx] = (sysdyn(xp, u, dt) - sysdyn(xn, u, dt)) / (2*eps)
    for du in range(Nu):
        up = u + eyeu[du] * eps
        un = u - eyeu[du] * eps
        B[:, du] = (sysdyn(x, up, dt) - sysdyn(x, un, dt)) / (2*eps)
    return A, B

In [61]:
# Following the notation as in the paper
def ilqr_iteration(u, x0, xf, Qf, Q, R, dt, sysdyn, eps=1e-5):
    du = np.zeros_like(u)
    N = len(u) + 1
    Nx = len(x0)
    Nu = u.shape[1]
    x = ilqr_openloop(x0, u, dt, sysdyn)
    dx = np.ones_like(x) * eps
    
    A = np.zeros((N-1, Nx, Nx))
    B = np.zeros((N-1, Nx, Nu))

    for k in range(N-1):
        A[k], B[k] = ilqr_linearization(x[k], u[k], dt, sysdyn)
    
    S = np.zeros((N, Nx, Nx))
    S[N-1] = Qf

    #nv = nx;
    v = np.zeros((N, Nx))
    v[N-1] = Qf.dot(x[N-1] - xf)

    K = np.zeros((N-1, Nu, Nx))
    Kv = np.zeros((N-1, Nu, Nx))
    Ku = np.zeros((N-1, Nu, Nu))
    
    for k in range(N-2, -1, -1):
        cominv = np.linalg.inv(B[k].T.dot(S[k+1]).dot(B[k]) + R)
        K[k] = cominv.dot(B[k].T).dot(S[k+1]).dot(A[k])
        Kv[k] = cominv.dot(B[k].T)
        Ku[k] = cominv.dot(R)
        
        S[k] = A[k].T.dot(S[k+1]).dot(A[k] - B[k].dot(K[k])) + Q
        v[k] = (A[k] - B[k].dot(K[k])).T.dot(v[k+1]) - K[k].T.dot(R).dot(u[k]) + Q.dot(x[k])
        du[k] = -K[k].dot(dx[k]) - Kv[k].dot(v[k+1]) - Ku[k].dot(u[k])

    return du

In [62]:

def ilqr(u0, x0, xf, Qf, Q, R, dt, n_itr, threshold, sysdyn, stepsize=0.25):
    # Run the ilqr algorithm using some initial state, nominal control signal,
    # and cost function.  iterate until the maximum iterations are reached
    # or until the change in the control signal goes below some threshold.
    #
    # u0: initial control sequence
    # x0: initial state
    # xf: desired goal state
    # Qf: final quadratic cost matrix on the state
    # Q: the quadratic integrated cost matrix on the state
    # R: the quadratic integrated cost matrix on the control
    # dt: the current time step used
    # n_itr: iterations the maximum number of iterations to run for
    # threshold: stop when the change in u goes below this scalar
    # sysdyn: dynamics of the system
    #
    # Return:
    # u: the optimized controls
    
    u = u0
    cost = ilqr_cost(u, x0, xf, Qf, Q, R, dt, sysdyn)

    for i in range(n_itr):
        du = ilqr_iteration(u, x0, xf, Qf, Q, R, dt, sysdyn)
        u = u + stepsize*du
        newcost = ilqr_cost(u, x0, xf, Qf, Q, R, dt, sysdyn)
        if abs(newcost - cost) < threshold:
            break
        cost = newcost
        print cost

    return u

In [63]:
u = np.zeros((120, 1))
x0 = np.array([np.pi, 0])
xf = np.array([0, 0])
Qf = np.diag([1, 1])
Q = np.zeros((2, 2))
R = 1e-5*np.eye(1)
dt = 5e-3
n_itr = 20
threshold = 0
new_u = ilqr(u, x0, xf, Qf, Q, R, dt, n_itr, threshold, pendulum_dynamics, stepsize=0.4)


155.39642736
85.8590238355
59.1782163465
2.26997933857
10.8769150186
4.55349717103
0.818303762594
0.567228097959
0.645492688455
0.562880718909
0.49256870438
0.471183786793
0.469315128747
0.469744423653
0.469450714919
0.469023409534
0.468802172186
0.468736844735
0.468727569294
0.468728098687


In [64]:
new_u

array([[-44.27788139],
       [-43.87665226],
       [-43.46467988],
       [-43.04207029],
       [-42.60893244],
       [-42.16537838],
       [-41.71152303],
       [-41.24748462],
       [-40.77338468],
       [-40.28934817],
       [-39.79550386],
       [-39.291984  ],
       [-38.77892529],
       [-38.25646818],
       [-37.72475779],
       [-37.18394344],
       [-36.63417912],
       [-36.0756242 ],
       [-35.50844202],
       [-34.93280184],
       [-34.34887742],
       [-33.75684795],
       [-33.15689751],
       [-32.54921558],
       [-31.93399671],
       [-31.31144025],
       [-30.68175034],
       [-30.04513637],
       [-29.40181137],
       [-28.75199314],
       [-28.09590329],
       [-27.43376681],
       [-26.7658124 ],
       [-26.09227089],
       [-25.41337581],
       [-24.72936252],
       [-24.0404678 ],
       [-23.34692891],
       [-22.6489835 ],
       [-21.94686894],
       [-21.24082139],
       [-20.53107532],
       [-19.81786326],
       [-19

In [80]:
from functools import partial

def flip(f):
    return lambda *args: f(*reversed(args))

def ilqr_forward_pass(x0, uref, sysdyn, cost_func, final_cost_func, K=None, xref=None):
    
    Dx = len(x0)
    N, Du = uref.shape
    x = np.zeros((N+1, Dx))
    uout = np.zeros((N, Du))
    cost = 0
    x[0] = x0
    for t in range(N):
        if K is not None and xref is not None:
            u = uref[t] + K[t].dot(x[t] - xref[t])
        else:
            u = uref[t]
        x[t+1] = sysdyn(x[t], u)
        cost += cost_func(x[t], u)
        uout[t] = u
    cost += final_cost_func(x[N])
    return x, cost, uout

In [81]:
def jacobian(x, f, eps=1e-5):
    Nx = len(x)
    Nf = len(f(x))
    J = np.zeros((Nf, Nx))
    eyex = np.eye(Nx)
    for dx in range(Nx):
        xp = x + eyex[dx] * eps
        xn = x - eyex[dx] * eps
        J[:, dx] = (f(xp) - f(xn)) / (2*eps)
    return J

def grad(x, f, eps=1e-5):
    Nx = len(x)
    g = np.zeros(Nx)
    eyex = np.eye(Nx)
    for dx in range(Nx):
        xp = x + eyex[dx] * eps
        xn = x - eyex[dx] * eps
        g[dx] = (f(xp) - f(xn)) / (2*eps)
    return g

def ilqr_linearize(x, u, sysdyn, cost_func, final_cost_func):
    
    Dx = x.shape[1]
    N, Du = u.shape
    fx = np.zeros((N, Dx, Dx))
    fu = np.zeros((N, Dx, Du))
    cx = np.zeros((N+1, Dx))
    cu = np.zeros((N, Du))
    cxx = np.zeros((N+1, Dx, Dx))
    cuu = np.zeros((N, Du, Du))
    cxu = np.zeros((N, Dx, Du))

    for k in range(N):
        fx[k] = jacobian(x[k], lambda x: sysdyn(x, u[k]))#partial(flip(sysdyn), u[k]))
        fu[k] = jacobian(u[k], lambda u: sysdyn(x[k], u))#partial(sysdyn, x[k]))
        cx[k] = grad(x[k], lambda x: cost_func(x, u[k]))#partial(flip(cost_func), u[k]))
        cu[k] = grad(u[k], lambda u: cost_func(x[k], u))#partial(cost_func, x[k]))
        cxu[k] = jacobian(u[k], lambda u: grad(x[k], lambda x: cost_func(x, u)))#partial(flip(cost_func), u)))
        cuu[k] = jacobian(u[k], lambda u: grad(u, lambda u: cost_func(x[k], u)))#partial(cost_func, x[k])))
    cx[N] = grad(x[N], final_cost_func)
    cxx[N] = jacobian(x[N], lambda x: grad(x, final_cost_func))
    return fx, fu, cx, cu, cxx, cxu, cuu

In [82]:
def ilqr_backward_pass(x, u, sysdyn, cost_func, final_cost_func):
    Dx = x.shape[1]
    N, Du = u.shape
    fx, fu, cx, cu, cxx, cxu, cuu = ilqr_linearize(x, u, sysdyn, cost_func, final_cost_func)
    
    Vx = np.zeros((N+1, Dx))
    Vxx = np.zeros((N+1, Dx, Dx))
    Vx[N] = cx[N]
    Vxx[N] = cxx[N]
    k = np.zeros((N, Du))
    K = np.zeros((N, Du, Dx))
    
    # dVx and dVxx are used for computing the expected reduction in cost
    # dV ~ -alpha*Q_u.T*k - alpha^2*k.T*Q_uu*k
    dVx = 0
    dVxx = 0
    
    
    for t in range(N-1, -1, -1):
        Qx = cx[t] + fx[t].T.dot(Vx[t+1])
        Qu = cu[t] + fu[t].T.dot(Vx[t+1])
        Qxx = cxx[t] + fx[t].T.dot(Vxx[t+1]).dot(fx[t])
        # TODO regularize
        Qux = cxu[t].T + fu[t].T.dot(Vxx[t+1]).dot(fx[t])
        Quu = cuu[t] + fu[t].T.dot(Vxx[t+1]).dot(fu[t])
        
        # TODO regularize
        k[t] = -np.linalg.inv(Quu)*Qu
        K[t] = -np.linalg.inv(Quu)*Qux
        
        Vx[t] = Qx - K[t].T.dot(Quu).dot(k[t])
        Vxx[t] = Qxx - K[t].T.dot(Quu).dot(K[t])
        
        dVx += np.inner(k[t], Qu)
        dVxx += 0.5*k[t].T.dot(Quu).dot(k[t])
    return Vx, Vxx, k, K, dVx, dVxx

In [83]:
def ilqr(x0, u0, sysdyn, cost_func, final_cost_func,
         max_iter=20,
         min_reduction=0,
         max_line_search_iter=8,
        ):
    
    Dx = len(x0)
    Du = u0.shape[1]
    N = len(u0)
    
    u = u0
    
    for itr in range(max_iter):
    
        x, cost, _ = ilqr_forward_pass(x0, u, sysdyn, cost_func, final_cost_func)
        print cost
        Vx, Vxx, k, K, dVx, dVxx = ilqr_backward_pass(x, u, sysdyn, cost_func, final_cost_func)

        alpha = 1
        for _ in range(max_line_search_iter):
            xnew, cnew, unew = ilqr_forward_pass(x0, u+alpha*k, sysdyn, cost_func, final_cost_func, K, x)
            dcost = cost - cnew
            expected = -alpha*(dVx+alpha*dVxx)
            if expected < 0:
                raise ValueError('should not happen')
            if dcost / expected > min_reduction:
                break
            else:
                alpha = alpha * 0.5
        u = unew
        if abs(cost - cnew) < 1e-6:
            print 'converged'
            break
        cost = cnew

u0 = np.zeros((120, 1))
x0 = np.array([np.pi, 0])
xf = np.array([0, 0])
Qf = np.diag([1, 1])
Q = np.zeros((2, 2))
R = 1e-5*np.eye(1)
dt = 5e-3
n_itr = 20
threshold = 0
    
ilqr(x0, u0, (lambda x, u: pendulum_dynamics(x, u, dt)), (lambda x, u: 0.5*x.T.dot(Q).dot(x) + 0.5*u.T.dot(R).dot(u)), (lambda x: 0.5*(x - xf).T.dot(Qf).dot(x - xf)))

4.93480220054
0.703262162678
0.468759154069
0.468726220699
converged
